In [ ]:
# Copyright 2023 Google LLC
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# Vertex AI 模型花园 MoViNet 视频剪辑分类

请参考以下链接：[在Colab中运行](https://colab.research.google.com/github/GoogleCloudPlatform/vertex-ai-samples/blob/main/notebooks/community/model_garden/model_garden_movinet_clip_classification.ipynb)，[在GitHub上查看](https://github.com/GoogleCloudPlatform/vertex-ai-samples/blob/main/notebooks/community/model_garden/model_garden_movinet_clip_classification.ipynb)，[在Vertex AI Workbench中打开](https://console.cloud.google.com/vertex-ai/notebooks/deploy-notebook?download_url=https://raw.githubusercontent.com/GoogleCloudPlatform/vertex-ai-samples/main/notebooks/community/model_garden/model_garden_movinet_clip_classification.ipynb)。

**_注意_**: 本笔记本已在以下环境中进行测试：

* Python版本 = 3.9

## 概述

本笔记本演示了如何在Vertex AI Model Garden中使用[MoViNet](https://github.com/tensorflow/models/tree/master/official/projects/movinet)。

### 目标

* 训练新模型
  * 将输入数据转换为训练格式
  * 创建[超参数调整作业](https://cloud.google.com/vertex-ai/docs/training/hyperparameter-tuning-overview)来训练新模型
  * 查找并导出最佳模型

* 测试已训练的模型
  * 将模型上传到[Vertex AI模型注册表](https://cloud.google.com/vertex-ai/docs/model-registry/introduction)
  * 运行批量预测

* 清理资源

### 成本

此教程使用Google Cloud的可计费组件：

* Vertex AI
* Cloud Storage

了解[Vertex AI定价](https://cloud.google.com/vertex-ai/pricing)和[Cloud Storage定价](https://cloud.google.com/storage/pricing)，并使用[Pricing计算器](https://cloud.google.com/products/calculator/)根据预期使用量生成成本估算。

在你开始之前

只有合作办公
如果您使用工作台，请跳过此部分并运行以下命令。

In [ ]:
import sys

if "google.colab" in sys.modules:
    ! pip3 install --upgrade google-cloud-aiplatform

    # Automatically restart kernel after installs
    import IPython

    app = IPython.Application.instance()
    app.kernel.do_shutdown(True)

    from google.colab import auth as google_auth

    google_auth.authenticate_user()

### 设置您的Google Cloud项目

**无论您使用什么笔记本环境，以下步骤都是必需的。**

1. [选择或创建一个Google Cloud项目](https://console.cloud.google.com/cloud-resource-manager)。当您第一次创建帐户时，您将获得300美元的免费信用额度，用于支付计算/存储成本。

1. [确保为您的项目启用了计费](https://cloud.google.com/billing/docs/how-to/modify-project)。

1. [启用Vertex AI API和Compute Engine API](https://console.cloud.google.com/flows/enableapi?apiid=aiplatform.googleapis.com,compute_component)。

1. 如果您将此笔记本本地运行，则需要安装[Cloud SDK](https://cloud.google.com/sdk)。

1. 在下面的单元格中输入您的项目ID。然后运行该单元格，以确保Cloud SDK在本笔记本中的所有命令中使用正确的项目。

1. [创建一个服务帐号](https://cloud.google.com/iam/docs/service-accounts-create#iam-service-accounts-create-console)，并为其分配`Vertex AI用户`和`存储对象管理员`角色，以便使用经过优化的模型运行批量预测。

**注意**：Jupyter会将以`!`前缀开头的行视为shell命令，并且会将以`$`前缀开头的Python变量插入这些命令中。

In [ ]:
import os

from google.cloud import aiplatform

# The GCP project ID for experiments.
PROJECT_ID = ""  # @param {type:"string"}

# Bucket URI with gs:// prefix.
BUCKET_URI = ""  # @param {type:"string"}

# You can choose a region from https://cloud.google.com/about/locations.
# Only regions prefixed by "us", "asia", or "europe" are supported.
REGION = "us-central1"  # @param {type:"string"}
REGION_PREFIX = REGION.split("-")[0]
assert REGION_PREFIX in (
    "us",
    "europe",
    "asia",
), f'{REGION} is not supported. It must be prefixed by "us", "asia", or "europe".'

! gcloud config set project $PROJECT_ID

STAGING_BUCKET = os.path.join(BUCKET_URI, "temporal")
CHECKPOINT_BUCKET = os.path.join(BUCKET_URI, "ckpt")

aiplatform.init(project=PROJECT_ID, location=REGION, staging_bucket=STAGING_BUCKET)

# Download config files.
CONFIG_DIR = os.path.join(BUCKET_URI, "config")

### 定义常量

In [ ]:
OBJECTIVE = "vcn"

# Data converter constants.
DATA_CONVERTER_JOB_PREFIX = "data_converter"
DATA_CONVERTER_CONTAINER = f"{REGION_PREFIX}-docker.pkg.dev/vertex-ai/vertex-vision-model-garden-dockers/data-converter"
DATA_CONVERTER_MACHINE_TYPE = "n1-highmem-8"

# Training constants.
TRAINING_JOB_PREFIX = "train"
TRAIN_CONTAINER_URI = f"{REGION_PREFIX}-docker.pkg.dev/vertex-ai/vertex-vision-model-garden-dockers/movinet-train"
TRAIN_MACHINE_TYPE = "n1-highmem-16"
TRAIN_ACCELERATOR_TYPE = "NVIDIA_TESLA_V100"
TRAIN_NUM_GPU = 2

# Evaluation constants.
EVALUATION_METRIC = "accuracy"

# Export constants.
EXPORT_JOB_PREFIX = "export"
EXPORT_CONTAINER_URI = f"{REGION_PREFIX}-docker.pkg.dev/vertex-ai/vertex-vision-model-garden-dockers/movinet-model-export"
EXPORT_MACHINE_TYPE = "n1-highmem-8"

# Prediction constants.
# You can adjust accelerator types and machine types to get faster predictions.
PREDICTION_CONTAINER_URI = f"{REGION_PREFIX}-docker.pkg.dev/vertex-ai/vertex-vision-model-garden-dockers/movinet-serve"
PREDICTION_PORT = 8080
PREDICTION_ACCELERATOR_COUNT = 1
PREDICTION_ACCELERATOR_TYPE = "NVIDIA_TESLA_T4"
PREDICTION_MACHINE_TYPE = "n1-standard-4"
PREDICTION_JOB_PREFIX = "predict"

### 定义常见的助手函数

In [ ]:
import json
from datetime import datetime
from typing import Any

import numpy as np
import tensorflow as tf
import yaml


def get_job_name_with_datetime(prefix: str) -> str:
    """Returns a timestamped job name with the given prefix."""
    return prefix + datetime.now().strftime("_%Y%m%d_%H%M%S")


def print_response_instance(json_str: str, label_map: dict[int, str]):
    """Prints summary of a prediction JSON result from the model response."""
    json_obj = json.loads(json_str)
    if "prediction" not in json_obj:
        print("Error:", json_str)
        return
    instance = json_obj["instance"]
    prediction = json_obj["prediction"]
    gcs_uri = instance["content"]
    time_start = instance.get("timeSegmentStart", "0.0s")
    time_end = instance.get("timeSegmentEnd", "Infinity")
    max_idx = np.argmax(prediction)
    print(f"{gcs_uri} {time_start}-{time_end}:", label_map[max_idx])


def get_label_map(label_map_yaml_filepath: str) -> tuple[dict[int, str], int]:
    """Reads label map from a YAML file and returns the label map with the number of classes."""
    with tf.io.gfile.GFile(label_map_yaml_filepath, "rb") as input_file:
        label_map = yaml.safe_load(input_file.read())["label_map"]
    num_classes = max(label_map.keys()) + 1
    return label_map, num_classes


def get_best_trial(
    model_di: str, max_trial_count: int, evaluation_metric: str
) -> tuple[str, Any]:
    """Finds the best trial directory and eval results from a hyperparameter tuning job."""
    best_trial_dir = ""
    best_trial_evaluation_results = {}
    best_performance = -1

    for i in range(max_trial_count):
        current_trial = i + 1
        current_trial_dir = os.path.join(model_dir, "trial_" + str(current_trial))
        current_trial_best_ckpt_dir = os.path.join(current_trial_dir, "best_ckpt")
        current_trial_best_ckpt_evaluation_filepath = os.path.join(
            current_trial_best_ckpt_dir, "info.json"
        )
        with tf.io.gfile.GFile(current_trial_best_ckpt_evaluation_filepath, "rb") as f:
            eval_metric_results = json.load(f)
            current_performance = eval_metric_results[evaluation_metric]
            if current_performance > best_performance:
                best_performance = current_performance
                best_trial_dir = current_trial_dir
                best_trial_evaluation_results = eval_metric_results
    return best_trial_dir, best_trial_evaluation_results


def find_checkpoint_in_dir(checkpoint_dir: str) -> str:
    """Finds a checkpoint path relative to the directory."""
    for root, dirs, files in tf.io.gfile.walk(checkpoint_dir):
        for file in files:
            if file.endswith(".index"):
                return os.path.join(root, os.path.splitext(file)[0])


def upload_checkpoint_to_gcs(checkpoint_url: str) -> str:
    """Uploads a compressed .tar.gz checkpoint at the given URL to Cloud Storage."""
    filename = os.path.basename(checkpoint_url)
    checkpoint_name = filename.replace(".tar.gz", "")
    print("Download checkpoint from", checkpoint_url, "and store to", CHECKPOINT_BUCKET)
    ! wget $checkpoint_url -O $filename
    ! mkdir -p $checkpoint_name
    ! tar -xvzf $filename -C $checkpoint_name

    checkpoint_path = find_checkpoint_in_dir(checkpoint_name)
    checkpoint_path = os.path.relpath(checkpoint_path, checkpoint_name)

    ! gsutil cp -r $checkpoint_name $CHECKPOINT_BUCKET/
    checkpoint_uri = os.path.join(CHECKPOINT_BUCKET, checkpoint_name, checkpoint_path)
    print("Checkpoint uploaded to", checkpoint_uri)
    return checkpoint_uri


def upload_config_to_gcs(url: str) -> str:
    """Uploads a config file at the given URL to Cloud Storage."""
    filename = os.path.basename(url)
    destination = os.path.join(CONFIG_DIR, filename)
    print("Copy", url, "to", destination)
    ! wget "$url" -O "$filename"
    ! gsutil cp "$filename" "$destination"
    return destination

## 训练新模型
本节介绍如何训练新模型。
1. 将输入数据转换为训练格式
2. 创建超参数调整作业以训练新模型
3. 寻找并导出最佳模型

如果您已经训练过模型，请转到部分`测试训练过的模型`。

请选择一个模型：
* `model_id`：MoViNet模型变体ID之一，可选择`a0`，`a1`，`a2`，`a3`，`a4`，`a5`。数字较大的模型需要更多资源来训练，并且预计具有更高的准确性和延迟。在这里，我们使用`a0`作为演示目的。
* `model_mode`：MoViNet模型类型，可以是`base`或者`stream`。基础模型具有稍高的准确性，而流式模型则经过优化，适用于流式传输和更快速的CPU推理。更多信息请参考[官方MoViNet文档](https://github.com/tensorflow/models/tree/master/official/projects/movinet)。

**注意**：目前预测容器仅支持基础模型（非流式）。如果您训练了流式模型，需要下载模型并参考[MoViNet官方指南](https://github.com/tensorflow/models/blob/master/official/projects/movinet/movinet_streaming_model_training_and_inference.ipynb)来在本地运行预测。

In [ ]:
model_id = "a0"  # @param ["a0", "a1", "a2", "a3", "a4", "a5"]
model_mode = "base"  # @param ["base", "stream"]
is_stream = model_mode == "stream"
model_name = f"movinet_{model_id}_{model_mode}"

if is_stream:
    export_container_args = {
        "conv_type": "2plus1d",
        "se_type": "2plus3d",
        "activation": "hard_swish",
        "gating_activation": "hard_sigmoid",
        "use_positional_encoding": model_id in {"a3", "a4", "a5"},
    }
else:
    export_container_args = {
        "conv_type": "3d",
        "se_type": "3d",
        "activation": "swish",
        "gating_activation": "sigmoid",
        "use_positional_encoding": False,
    }

### 为训练准备输入数据

按照[此处](https://cloud.google.com/vertex-ai/docs/video-data/classification/prepare-data)描述的格式准备数据，然后通过运行下面的代码单元将其转换为训练格式：

* `input_file_path`：准备数据的输入文件路径。
* `input_file_type`：输入文件类型，如`csv`或`jsonl`。
* `output_fps`：视频的采样率；每秒帧数。
* `split_ratio`：三个逗号分隔的浮点数，表示要拆分为训练/验证/测试数据的比例。它们的总和必须为1。
* `num_shard`：三个逗号分隔的整数，表示用于训练/验证/测试的分片数量。

In [ ]:
# This job will convert input data as training format, with given split ratios
# and number of shards on train/test/validation.

data_converter_job_name = get_job_name_with_datetime(
    DATA_CONVERTER_JOB_PREFIX + "_" + OBJECTIVE
)

input_file_path = ""  # @param {type:"string"}
input_file_type = "csv"  # @param ["csv", "jsonl"]
output_fps = 5  # @param {type:"integer"}
split_ratio = "0.8,0.1,0.1"
num_shard = "10,10,10"
data_converter_output_dir = os.path.join(BUCKET_URI, data_converter_job_name)


worker_pool_specs = [
    {
        "machine_spec": {
            "machine_type": DATA_CONVERTER_MACHINE_TYPE,
        },
        "replica_count": 1,
        "container_spec": {
            "image_uri": DATA_CONVERTER_CONTAINER,
            "command": [],
            "args": [
                "--input_file_path=%s" % input_file_path,
                "--input_file_type=%s" % input_file_type,
                "--objective=%s" % OBJECTIVE,
                "--num_shard=%s" % num_shard,
                "--split_ratio=%s" % split_ratio,
                "--output_dir=%s" % data_converter_output_dir,
                "--output_fps=%d" % output_fps,
            ],
        },
    }
]

data_converter_custom_job = aiplatform.CustomJob(
    display_name=data_converter_job_name,
    project=PROJECT_ID,
    worker_pool_specs=worker_pool_specs,
    staging_bucket=STAGING_BUCKET,
)

data_converter_custom_job.run()

input_train_data_path = os.path.join(data_converter_output_dir, "train.tfrecord*")
input_validation_data_path = os.path.join(data_converter_output_dir, "val.tfrecord*")
label_map_path = os.path.join(data_converter_output_dir, "label_map.yaml")
print("input_train_data_path for training: ", input_train_data_path)
print("input_validation_data_path for training: ", input_validation_data_path)
print("label_map_path for prediction: ", label_map_path)

使用Vertex AI SDK创建并运行具有Vertex AI Model Garden训练docker镜像的[超参数调整作业](https://cloud.google.com/vertex-ai/docs/training/hyperparameter-tuning-overview)。

#### 定义以下规格

* `worker_pool_specs`：一系列指定机器类型和docker镜像的字典。此示例定义一个具有一个`n1-highmem-16`机器和2个`NVIDIA_TESLA_V100` GPU的单节点集群。

  **注意**：我们建议为MoViNet-A2和更大模型使用8个GPU。由于加载视频数据需要大量的GPU内存，建议首先尝试小批量大小进行实验。
* `parameter_spec`：指定要优化的参数的字典。字典键是您的训练应用代码中每个超参数的命令行参数分配的字符串，字典值是参数规范。参数规范包括超参数的类型、最小/最大值和比例。
* `metric_spec`：指定要优化的指标的字典。字典键是您在训练应用代码中设置的`hyperparameter_metric_tag`，值是优化目标。

In [ ]:
from google.cloud.aiplatform import hyperparameter_tuning as hpt

# Input train and validation datasets can be found from the section above
# `Prepare input data for training`.
# Or, set prepared datasets paths if already exist.
# input_train_data_path = ""
# input_validation_data_path = ""
# label_map_path = ""

train_job_name = get_job_name_with_datetime(f"{TRAINING_JOB_PREFIX}_{model_name}")
model_dir = os.path.join(BUCKET_URI, train_job_name)
label_map, num_classes = get_label_map(label_map_path)

# Uploads pretained checkpoint to GCS bucket.
init_checkpoint = f"https://storage.googleapis.com/tf_model_garden/vision/movinet/{model_name}_with_backbone.tar.gz"
init_checkpoint = upload_checkpoint_to_gcs(init_checkpoint)

# Uploads config file according to model_id and streaming options.
config_file = f"{model_id}_stream" if is_stream else model_id
config_file = f"https://raw.githubusercontent.com/tensorflow/models/master/official/projects/movinet/configs/yaml/movinet_{config_file}_gpu.yaml"
config_file = upload_config_to_gcs(config_file)

# The parameters here are mainly for demonstration purpose. Please update them
# for better performance.
trainer_args = {
    "experiment": "movinet_kinetics600",
    "config_file": config_file,
    "input_train_data_path": input_train_data_path,
    "input_validation_data_path": input_validation_data_path,
    "init_checkpoint": init_checkpoint,
    "model_dir": model_dir,
    "num_classes": num_classes,
    "global_batch_size": 4,
    "prefetch_buffer_size": 8,
    "shuffle_buffer_size": 32,
    "train_steps": 2000,
}

worker_pool_specs = [
    {
        "machine_spec": {
            "machine_type": TRAIN_MACHINE_TYPE,
            "accelerator_type": TRAIN_ACCELERATOR_TYPE,
            # Each training job uses TRAIN_NUM_GPU GPUs.
            "accelerator_count": TRAIN_NUM_GPU,
        },
        "replica_count": 1,
        "container_spec": {
            "image_uri": TRAIN_CONTAINER_URI,
            "args": [
                "--mode=train_and_eval",
                "--params_override=runtime.num_gpus=%d" % TRAIN_NUM_GPU,
            ]
            + ["--{}={}".format(k, v) for k, v in trainer_args.items()],
        },
    }
]

metric_spec = {"model_performance": "maximize"}

# These learning rates might not be optimal for your selected model type; To
# tune learning rates, try hpt.DoubleParameterSpec with more trials.
LEARNING_RATES = [1e-3, 3e-3]
MAX_TRIAL_COUNT = len(LEARNING_RATES)
parameter_spec = {
    "learning_rate": hpt.DiscreteParameterSpec(values=LEARNING_RATES, scale="linear"),
}

print(worker_pool_specs, metric_spec, parameter_spec)

#### 运行超参数调优作业
* `max_trial_count`: 设置服务将运行的试验次数上限。建议的做法是从较小的试验次数开始，了解您选择的超参数对结果的影响程度，然后再逐步增加试验次数。

* `parallel_trial_count`: 如果您使用并行试验，服务将为多个训练处理集群提供资源。在创建作业时指定的工作池规范将用于每个独立的训练集群。增加并行试验的数量会减少超参数调优作业运行的时间；但是，这可能会降低整体作业的效果。这是因为默认的调优策略使用先前试验的结果来指导后续试验中值的指定。

* `search_algorithm`: 可用的搜索算法有网格算法、随机算法或默认算法（None）。默认选项应用贝叶斯优化来搜索可能超参数值的空间，并且是推荐的算法。

单击输出中生成的链接以在Cloud Console中查看您的运行。

In [ ]:
train_custom_job = aiplatform.CustomJob(
    display_name=train_job_name,
    project=PROJECT_ID,
    worker_pool_specs=worker_pool_specs,
    staging_bucket=STAGING_BUCKET,
)

train_hpt_job = aiplatform.HyperparameterTuningJob(
    display_name=train_job_name,
    custom_job=train_custom_job,
    metric_spec=metric_spec,
    parameter_spec=parameter_spec,
    max_trial_count=MAX_TRIAL_COUNT,
    parallel_trial_count=MAX_TRIAL_COUNT,
    project=PROJECT_ID,
    search_algorithm=None,
)

train_hpt_job.run()

print("model_dir is:", model_dir)

### 以Tensorflow SavedModel格式导出模型

In [ ]:
# This job will export models from TF checkpoints to TF saved model format.
# model_dir is from the section above.
best_trial_dir, best_trial_evaluation_results = get_best_trial(
    model_dir, MAX_TRIAL_COUNT, EVALUATION_METRIC
)
best_checkpoint_path = find_checkpoint_in_dir(f"{best_trial_dir}/best_ckpt/")
print("best_trial_dir: ", best_trial_dir)
print("best_trial_evaluation_results: ", best_trial_evaluation_results)
print("best_checkpoint: ", best_checkpoint_path)

container_args = {
    "export_path": f"{model_dir}/best_model",
    "model_id": model_id,
    "num_classes": num_classes,
    "causal": is_stream,
    "checkpoint_path": best_checkpoint_path,
    "assert_checkpoint_objects_matched": False,
    **export_container_args,
}

worker_pool_specs = [
    {
        "machine_spec": {
            "machine_type": EXPORT_MACHINE_TYPE,
        },
        "replica_count": 1,
        "container_spec": {
            "image_uri": EXPORT_CONTAINER_URI,
            "args": ["--{}={}".format(k, v) for k, v in container_args.items()],
        },
    }
]

model_export_job_name = get_job_name_with_datetime(EXPORT_JOB_PREFIX + "_" + OBJECTIVE)
model_export_custom_job = aiplatform.CustomJob(
    display_name=model_export_job_name,
    project=PROJECT_ID,
    worker_pool_specs=worker_pool_specs,
    staging_bucket=STAGING_BUCKET,
)

model_export_custom_job.run()

print("best model is saved to: ", container_args["export_path"])

## 测试训练好的模型
本节展示了如何使用训练好的模型进行测试。
1. 将模型上传并部署到[Vertex AI模型注册表](https://cloud.google.com/vertex-ai/docs/model-registry/introduction)
2. 运行批量预测

**注意：** 预测容器仅适用于基本模型。如果您训练了一个流模型，请下载模型并参考[MoViNet官方指南](https://github.com/tensorflow/models/blob/master/official/projects/movinet/movinet_streaming_model_training_and_inference.ipynb)来在本地进行预测。

### 上传模型到Vertex AI模型注册表

以下单元格将训练好的模型上传到Vertex AI模型注册表。如果您想在已上传的模型上运行批处理预测，则可以跳过此步骤。

#### 可配置的环境变量

* `MODEL_PATH`：MoViNet模型的Cloud Storage URI。
* `BATCH_SIZE`：推理批处理大小。使用较大的值可以加速GPU预测。
* `NUM_FRAMES`：使用该模型进行单次预测时的帧数。
* `FPS`：每秒视频采样帧数。
* `OVERLAP_FRAMES`：允许连续预测窗口之间重叠帧数。设置较小的值可加快推理速度，但精度较低。

In [ ]:
serving_env = {
    "MODEL_ID": "tfvision-movinet-vcn",
    "MODEL_PATH": container_args["export_path"],
    "BATCH_SIZE": 1,  # Select a larger batch size to accelerate GPU prediction.
    "NUM_FRAMES": 32,
    "FPS": output_fps,
    "OVERLAP_FRAMES": 24,
    "OBJECTIVE": OBJECTIVE,
    "DEPLOY_SOURCE": "notebook",
}

model = aiplatform.Model.upload(
    display_name=model_name,
    serving_container_image_uri=PREDICTION_CONTAINER_URI,
    serving_container_ports=[PREDICTION_PORT],
    serving_container_predict_route="/predict",
    serving_container_health_route="/ping",
    serving_container_environment_variables=serving_env,
)

model.wait()

print("The uploaded model name is: ", model_name)

或者，取消以下单元格的注释以使用已上传的模型。将模型名称字符串替换为现有模型的名称。

In [ ]:
# model = aiplatform.Model("projects/123456789/locations/us-central1/models/12345678901234567890")

### 运行批量预测

我们现在将使用训练好的MoViNet剪辑分类模型进行批量预测，使用[Vertex AI 批量预测](https://cloud.google.com/vertex-ai/docs/predictions/get-batch-predictions)。

请准备一个输入的JSONL文件，每行遵循[这种格式](https://cloud.google.com/vertex-ai/docs/video-data/classification/get-predictions?hl=en#input_data_requirements)，并将其存储在Cloud Storage存储桶中。该服务帐户应具有访问包含训练模型和输入数据的存储桶的读取权限。有关更多信息，请参阅[服务帐户概述](https://cloud.google.com/iam/docs/service-account-overview)。

In [ ]:
# Path to the prediction input JSONL file.
test_jsonl_path = ""  # @param {type:"string"}
# Full service account name with the suffix `gserviceaccount.com`.
batch_predict_service_account = ""  # @param {type:"string"}

predict_job_name = get_job_name_with_datetime(f"{PREDICTION_JOB_PREFIX}_{model_name}")
predict_destination_prefix = os.path.join(STAGING_BUCKET, predict_job_name)

batch_prediction_job = model.batch_predict(
    job_display_name=predict_job_name,
    gcs_source=test_jsonl_path,
    gcs_destination_prefix=predict_destination_prefix,
    machine_type=PREDICTION_MACHINE_TYPE,
    accelerator_count=PREDICTION_ACCELERATOR_COUNT,
    accelerator_type=PREDICTION_ACCELERATOR_TYPE,
    max_replica_count=1,
    service_account=batch_predict_service_account,
)

batch_prediction_job.wait()

print(batch_prediction_job.display_name)
print(batch_prediction_job.resource_name)
print(batch_prediction_job.state)

您可以在输出目录中读取预测响应的JSONL文件。

In [ ]:
# The label map file was generated from the section above (`Prepare input data for training`).
for file in tf.io.gfile.glob(os.path.join(predict_destination_prefix, "*/*")):
    with tf.io.gfile.GFile(file, "r") as f:
        for line in f:
            print_response_instance(line, label_map)

清理

In [ ]:
# Delete the trained model.
model.delete()
# Delete custom and hpt jobs.
if data_converter_custom_job.list(filter=f'display_name="{data_converter_job_name}"'):
    data_converter_custom_job.delete()
if train_hpt_job.list(filter=f'display_name="{train_job_name}"'):
    train_hpt_job.delete()
if model_export_custom_job.list(filter=f'display_name="{model_export_job_name}"'):
    model_export_custom_job.delete()
if batch_prediction_job.list(filter=f'display_name="{predict_job_name}"'):
    batch_prediction_job.delete()